In [1]:
import os,sys,tqdm
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import *

from collections import Counter
import pandas as pd
import shutil
import pickle
import gc
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

In [3]:
max_words = 8352#8352 #Top most frequent words to consider. Any less frequent word will appear as oov_char value in the sequence data.
max_length = 360#360

In [4]:
word_index = reuters.get_word_index()
print('all_words#:',len(word_index))
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,maxlen=max_length,
                                                         test_split=0.2,seed=830913)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

all_words#: 30979
8260 train sequences
2066 test sequences


In [13]:
# train_len = [len(x) for x in x_train]
# test_len = [len(x) for x in x_test]
# all_len = train_len
# all_len.extend(test_len)
# basic_statistics(all_len)

,length
mean,145.964197
std,145.878476
mode,17.000000
min,2.000000
q1,60.000000
median,95.000000
q3,180.000000
max,2376.000000
iqr,120.000000
outlier,360.000000


In [15]:
# df = pd.DataFrame(all_len)
# df.to_excel('./results/length_dist.xlsx', header=False, index=False)
# df

In [21]:
# train_words = []
# for x in x_train:
#     train_words.extend(x)
# test_words = []
# for x in x_test:
#     test_words.extend(x)
# all_words = train_words
# all_words.extend(test_words)
# all_statistcs = Counter(all_words)
# all_statistcs

Counter({1: 11228,
         53: 4213,
         352: 647,
         26: 8451,
         14: 15015,
         279: 801,
         39: 5818,
         72: 3091,
         4497: 26,
         18: 11039,
         83: 2597,
         5291: 21,
         88: 2381,
         5397: 20,
         11: 20141,
         3412: 37,
         19: 10755,
         151: 1363,
         230: 962,
         831: 253,
         15: 13329,
         165: 1232,
         318: 707,
         3780: 33,
         124: 1676,
         1527: 117,
         1424: 128,
         35: 6588,
         5302: 20,
         12: 16668,
         17: 11191,
         486: 459,
         341: 663,
         142: 1466,
         255: 870,
         219: 997,
         429: 528,
         68: 3363,
         146: 1402,
         252: 882,
         191: 1098,
         15448: 3,
         3631: 35,
         2283: 65,
         71: 3120,
         10: 29581,
         342: 660,
         49: 4565,
         1977: 80,
         324: 695,
         27: 8311,
         9222: 

In [28]:
# df = pd.DataFrame.from_dict(dict(all_statistcs), orient = 'index')
# df.to_excel('./results/words_dist2.xlsx', header=False, index=True)
# df

,0
1,11228
53,4213
352,647
26,8451
14,15015
...,...
24452,1
18567,2
27222,1
26864,1


In [5]:
trainX = tf.keras.preprocessing.sequence.pad_sequences(x_train,maxlen=max_length,padding='post',value=0)
testX = tf.keras.preprocessing.sequence.pad_sequences(x_test,maxlen=max_length,padding='post',value=0)
print(trainX.shape, testX.shape)

(8260, 360) (2066, 360)


In [563]:
hidden_dim = 128
do = 0.1

## Graph execution
### Embedder

In [42]:
int_id = Input(shape=(max_length,), dtype='int32', name='int_ids') # 輸入的api funvtion name ID
int_ids = Masking(mask_value=0)(int_id)
sent_emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')(int_ids) 

### Encoder

In [43]:
rnn = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)(sent_emb)
rnn = BatchNormalization(name='bn')(rnn)


### Filter

In [44]:
fil = TimeDistributed(Dense(1,activation='sigmoid',
                             name='filter_out'),name='TD2')(rnn)

### Classfier

In [47]:
mul = Multiply()([fil,sent_emb])
clf = LSTM(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm')(mul)
clf = BatchNormalization(name='bn3')(clf)
clf = Dense(max(y_train)+1,activation='softmax',name='clf')(clf)

## Compile

In [48]:
model = Model(inputs=int_id, outputs = clf)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
int_ids (InputLayer)            [(None, 358)]        0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 358)          0           int_ids[0][0]                    
__________________________________________________________________________________________________
glove_emb (Embedding)           (None, 358, 128)     1280000     masking_1[0][0]                  
__________________________________________________________________________________________________
common_extract (GRU)            (None, 358, 64)      37248       glove_emb[0][0]                  
______________________________________________________________________________________________

In [53]:
# loss
import keras.backend as K
def custom_objective(layer):
    return K.sum(layer.output)
#     return K.sum(layer.output)
# kk = tf.keras.backend.ea
model.compile(loss=custom_objective(model.get_layer(name='TD2')),optimizer='adam')

Using TensorFlow backend.


OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

## Eager Execution

In [52]:
# whole model
do = 0
init = tensorflow.keras.initializers.Ones()
class base_model(Model):
    def __init__(self):
        super(base_model, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.bn1 = BatchNormalization(name='bn1')
        self.fil = Dense(1,activation='hard_sigmoid',kernel_initializer=init,bias_initializer=init,name='filter_out')
        #self.fil = TimeDistributed(Dense(1,activation='sigmoid', name='filter_out'),name='TD2')
        self.mul = Multiply()
        self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = LSTM(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')
    def transform(self,x):
        return tf.math.round(x)
    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.rnn1(x1)
        x = self.bn1(x)
        y = self.fil(x)
        y1 = self.transform(y)
        x2 = self.mul([y1,x1])
        x = self.rnn2(x2) #x
        x = self.bn2(x)
        y2 = self.out(x)
        return y,y1,y2
        #return y,y1,y2,x2
        
model = base_model()

In [691]:
# partial1 model
init_w = tensorflow.keras.initializers.Constant(value=0.9) #portyion=0.6, w=0.9, b = 0.8-0.85
init_b = tensorflow.keras.initializers.Constant(value=0.83) #w=1 ; b=0.499, portion=1
def onezero(x):
    portion = 0.6#0.6#0.6 #0.6~1
    z = tf.where(x>=1.0, x - x + 1.0, x)
    y = tf.where(z<=0.0, z - z + 0.0, portion*z)
    return y

class base_model_1(Model):
    def __init__(self):
        super(base_model_1, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        #self.rnn1 = GRU(int(hidden_dim/4),return_sequences=True,return_state=False,name='common_extract'
                      #,trainable=True)
        self.att = Attention(name='selfatt')
        #self.bn1 = BatchNormalization(name='bn1')
        #self.fil = Dense(1,activation=onezero,name='filter_out')
        self.fil = TimeDistributed(Dense(1,activation=onezero,kernel_initializer=init_w,bias_initializer=init_b, name='filter_out'),name='TD2') #relu/linear/step function

    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.att([x1,x1])
        #x = self.rnn1(x1)
        #x = self.bn1(x)
        y = self.fil(x)
        return x1,y

model1 = base_model_1()
#phase2
# model3 = load_model(saveP)

# model3 = base_model_1()
# model3.load_weights(saveP)#,by_name=True)
# model1.emb.set_weights(model3.emb.get_weights())

In [692]:
# partial2 model
class base_model_2(Model):
    def __init__(self):
        super(base_model_2, self).__init__()
        self.mul = Multiply()
        self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = GRU(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')

    def call(self,x1,y1):
        x2 = self.mul([y1,x1])
        x = self.rnn3(x2) #x2 #y1=weight|binary
        x = self.bn2(x)
        y2 = self.out(x)
        return y2
    
model2 = base_model_2()

In [448]:
'''
# TEST
x = tf.random.uniform((32, 6))
out1,out2 = model1(x)
out2 = tf.squeeze(out2,axis=-1)
# out2 = out2.astype('float32')
# out2 = tf.dtypes.cast(out2, tf.float8)
# print(out2.shape)
yy = tf.matmul(out2,kk)
print(yy[0].shape)
tf.where(yy[0]==0,1,0)

gg = tf.range(-5,5)
gg = tf.expand_dims(gg,axis=0)
gg = tf.keras.backend.repeat_elements(gg,rep=32,axis=0)
tf.where(gg==1,gg,0)

kk = tf.Variable(np.array([[1.0,1.0,1.0,0.0,0.0,0.0],[0.0,1.0,1.0,1.0,0.0,0.0],[0.0,0.0,1.0,1.0,1.0,0.0],
                           [0.0,0.0,0.0,1.0,1.0,1.0]]).T,dtype='float32')
kk = tf.expand_dims(kk,axis=0)
kk = tf.keras.backend.repeat_elements(kk,rep=1,axis=0)
kk.shape
'''

(32, 4)


<tf.Tensor: id=5770753, shape=(32, 4), dtype=int32, numpy=
array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]], dtype=int32)>

In [693]:
batch_size = 128 #,reshuffle_each_iteration=True
train_ds = tf.data.Dataset.from_tensor_slices((trainX,y_train)).shuffle(trainX.shape[0]).batch(batch_size)
valid_ds = tf.data.Dataset.from_tensor_slices((testX,y_test)).batch(batch_size)

In [694]:
seq_num = 4 #連續幾個才叫做連續

arr_len = max_length - seq_num + 1
seq_arr = []
for i in range(arr_len):
    ori_np = np.array([0]*max_length)
    modify_idx = [i,i+1,i+2,i+3]
    ori_np[modify_idx]=1
    seq_arr.append(ori_np)
seq_arr = np.array(seq_arr)
seq_mask = tf.Variable(seq_arr.T,dtype='float32')
seq_mask = tf.expand_dims(seq_mask,axis=0)
seq_mask

<tf.Tensor: id=7832025, shape=(1, 360, 357), dtype=float32, numpy=
array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)>

In [695]:
def loss_object1(predictions): #filter loss
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def one_percentage(predictions): #1 num
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def seq_loss(predictions):
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    predictions = tf.squeeze(predictions,axis=-1)
    results = tf.matmul(predictions,seq_mask)
    results = tf.where(results==seq_num,1.0,0.0)
    loss_ = tf.reduce_mean(results)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy() #clf loss

optimizer1 = tf.keras.optimizers.Nadam()
optimizer2 = tf.keras.optimizers.RMSprop()

train_loss = tf.keras.metrics.Mean(name='train_loss') #total_loss
train_accloss = tf.keras.metrics.Mean(name='train_accloss')#loss_acc
train_filloss = tf.keras.metrics.Mean(name='train_filloss') #loss_filter
train_seqloss = tf.keras.metrics.Mean(name='train_seqloss') #loss_seq
train_ones = tf.keras.metrics.Mean(name='train_ones') #ones_num
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') #acc_rate

test_loss = tf.keras.metrics.Mean(name='test_loss') #total_loss
test_accloss = tf.keras.metrics.Mean(name='test_accloss')#loss_acc
test_filloss = tf.keras.metrics.Mean(name='test_filloss') #loss_filter
test_seqloss = tf.keras.metrics.Mean(name='test_seqloss')
test_ones = tf.keras.metrics.Mean(name='test_ones') #ones_num
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [696]:
# seperate partial model
alpha = 0.0 #pahse1: -0.1 / 0.0 ; phase2: 0.01~0.05~0.1 有1-就是希望1越多，沒1-就是希望0越多1越少
beta = 1.0 #clf loss 越大越要求分好
gamma = 0.0 #seqloss 越大越要求連續
#-0.001 / 1.0 / 1.0

@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=False) as tape:
        emb, pred_imp = model1(x)
        #loss1 = alpha*loss_object1(pred_imp) #phase1
        #pred_imp2 = tf.math.round(pred_imp)
        #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
        pred_imp2 = tf.math.round(pred_imp)
        loss1 = loss_object1(pred_imp) #有1-就是希望1越多，沒1-就是希望0越多1越少 #pahse2:alpha*loss_object1(pred_imp) ; phase1: alpha*(1-loss_object1(pred_imp))
        pred_cat = model2(emb,pred_imp2) #pahse1: pred_imp; phase2; pred_imp2
        loss2 = loss_object2(yc, pred_cat)
        loss3 = 1-seq_loss(pred_imp2)
        loss = alpha*loss1 + beta*loss2 + gamma*loss3
    trainable_variable = model1.trainable_variables
    trainable_variable.extend(model2.trainable_variables)
    gradients = tape.gradient(loss,trainable_variable)
    optimizer1.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss) #total_loss
    train_filloss(loss1)
    train_accloss(loss2)
    train_seqloss(loss3) #loss_seq
    train_accuracy(yc, pred_cat) #acc_rate
    ones = one_percentage(pred_imp2) #pred_imp2
    train_ones(ones) #ones_num
    
    
@tf.function
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #phase1: pred_imp ; phase2:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)
    

In [331]:
'''
#AIO
alpha = 0.1
beta = 1
gamma = 0
@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=True) as tape: #persistent=True
        pred_imp,pred_round , pred_cat = model(x)
#         pred_cat = model(x)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
        loss1 = alpha*loss_object1(pred_imp)
        loss2 = beta*loss_object2(yc,pred_cat)
        loss = loss_object2(yc,pred_cat)
#     gradients = tape.gradient(loss, model.trainable_variables)
    grad1 = tape.gradient(loss1, model.trainable_variables)
    grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer1.apply_gradients(zip(gradients, model.trainable_variables))
    optimizer1.apply_gradients(zip(grad1, model.trainable_variables))
    optimizer2.apply_gradients(zip(grad2, model.trainable_variables))
#     with tf.GradientTape() as tape:
#         pred_imp , pred_cat = model(x)
#         loss2 = loss_object2(yc,pred_cat)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
#     grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer2.apply_gradients(zip(grad2, model.trainable_variables))

    train_loss(loss)
    train_accuracy(yc, pred_cat)
    ones = one_percentage(pred_round)
    train_ones(ones)
    
@tf.function
def test_step(x,yc):
    pred_imp,pred_round, pred_cat = model(x)
#     pred_cat = model(x)
    t_loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat) 
#     t_loss = loss_object2(yc,pred_cat)
    
    test_loss(t_loss)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_round)
    test_ones(t_ones)
'''    

In [697]:
EPOCHS = 2000
saveP = './model/emb_layer_weights'
train_loss_acc = []
train_loss_filter = []
train_loss_seq = []
train_weighted_loss = []
train_acc_rate = []
train_ones_num = []

test_loss_acc = []
test_loss_filter = []
test_loss_seq = []
test_weighted_loss = []
test_acc_rate = []
test_ones_num = []

# signature_dict = {'att':model1.att}

gc.collect()
best_clf = 0.0
for epoch in range(EPOCHS):
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        test_step(test_text, test_labels)

    template = 'Epoch {}, Total Loss: {}, Clf Loss: {}, Filter Loss: {}, Seq Loss: {}, Accuracy Rate: {:5.2f}%, Ones Portion: {}, \
            Test_Total_Loss: {}, Test_Clf_Loss: {}, Test_Filter_Loss: {}, TEST_Seq_Loss: {}, Test_Accuracy_Rate: {:5.2f}%, Test_Ones_Portion: {}'
    print(template.format(epoch+1,train_loss.result(),
                          train_accloss.result(),train_filloss.result(),train_seqloss.result(),
                        train_accuracy.result()*100,train_ones.result(),
                        test_loss.result(),
                        test_accloss.result(),test_filloss.result(),test_seqloss.result(),
                        test_accuracy.result()*100,test_ones.result(),
                        ))

    train_loss_acc.append(train_accloss.result())
    train_loss_filter.append(train_filloss.result())
    train_loss_seq.append(train_seqloss.result())
    train_weighted_loss.append(train_seqloss.result())
    train_acc_rate.append(train_accuracy.result()*100)
    train_ones_num.append(train_ones.result())
    
    test_loss_acc.append(test_accloss.result())
    test_loss_filter.append(test_filloss.result())
    test_loss_seq.append(test_seqloss.result())
    test_weighted_loss.append(test_loss.result())
    test_acc_rate.append(test_accuracy.result()*100)
    test_ones_num.append(test_ones.result())
    if best_clf<=train_accuracy.result()*100:
        #tf.saved_model.save(model1,saveP)
        model1.save_weights(saveP,save_format='tf')
        best_clf = train_accuracy.result()*100
        print('===MODEL1 WEIGHTS SAVED===',saveP)
    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accloss.reset_states()
    train_filloss.reset_states()
    train_seqloss.reset_states()
    train_ones.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accloss.reset_states()
    test_filloss.reset_states()
    test_seqloss.reset_states()
    test_ones.reset_states()
    test_accuracy.reset_states()

Epoch 1, Total Loss: 2.658255100250244, Clf Loss: 2.658255100250244, Filter Loss: 0.5948817133903503, Seq Loss: 0.00691008847206831, Accuracy Rate: 36.19%, Ones Portion: 0.9931657314300537,             Test_Total_Loss: 2.366525650024414, Test_Clf_Loss: 2.366525650024414, Test_Filter_Loss: 0.5945746302604675, TEST_Seq_Loss: 0.012231027707457542, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 0.9879305362701416
===MODEL1 WEIGHTS SAVED=== ./model/emb_layer_weights
Epoch 2, Total Loss: 2.3358864784240723, Clf Loss: 2.3358864784240723, Filter Loss: 0.595204770565033, Seq Loss: 0.011633838526904583, Accuracy Rate: 37.72%, Ones Portion: 0.9889929890632629,             Test_Total_Loss: 2.2806904315948486, Test_Clf_Loss: 2.2806904315948486, Test_Filter_Loss: 0.5916910767555237, TEST_Seq_Loss: 0.048530250787734985, Test_Accuracy_Rate: 37.32%, Test_Ones_Portion: 0.9570118188858032
===MODEL1 WEIGHTS SAVED=== ./model/emb_layer_weights
Epoch 3, Total Loss: 2.066739797592163, Clf Loss: 2.066739797592

Epoch 19, Total Loss: 1.4378191232681274, Clf Loss: 1.4378191232681274, Filter Loss: 0.570634663105011, Seq Loss: 0.22471559047698975, Accuracy Rate: 65.02%, Ones Portion: 0.8136064410209656,             Test_Total_Loss: 1.7626128196716309, Test_Clf_Loss: 1.7626128196716309, Test_Filter_Loss: 0.5647392272949219, TEST_Seq_Loss: 0.2563219368457794, Test_Accuracy_Rate: 57.70%, Test_Ones_Portion: 0.7819822430610657
===MODEL1 WEIGHTS SAVED=== ./model/emb_layer_weights
Epoch 20, Total Loss: 1.3862950801849365, Clf Loss: 1.3862950801849365, Filter Loss: 0.5670374035835266, Seq Loss: 0.24468348920345306, Accuracy Rate: 66.21%, Ones Portion: 0.7945711016654968,             Test_Total_Loss: 1.7487764358520508, Test_Clf_Loss: 1.7487764358520508, Test_Filter_Loss: 0.561374306678772, TEST_Seq_Loss: 0.27272602915763855, Test_Accuracy_Rate: 58.23%, Test_Ones_Portion: 0.7664234638214111
===MODEL1 WEIGHTS SAVED=== ./model/emb_layer_weights
Epoch 21, Total Loss: 1.339896321296692, Clf Loss: 1.3398963212

Epoch 37, Total Loss: 1.092486023902893, Clf Loss: 1.092486023902893, Filter Loss: 0.5478132963180542, Seq Loss: 0.364610880613327, Accuracy Rate: 72.65%, Ones Portion: 0.6797308921813965,             Test_Total_Loss: 1.6842490434646606, Test_Clf_Loss: 1.6842490434646606, Test_Filter_Loss: 0.5404362082481384, TEST_Seq_Loss: 0.39362603425979614, Test_Accuracy_Rate: 60.79%, Test_Ones_Portion: 0.6486999988555908
Epoch 38, Total Loss: 1.096957802772522, Clf Loss: 1.096957802772522, Filter Loss: 0.5461334586143494, Seq Loss: 0.37315717339515686, Accuracy Rate: 72.54%, Ones Portion: 0.6716547608375549,             Test_Total_Loss: 1.7027912139892578, Test_Clf_Loss: 1.7027912139892578, Test_Filter_Loss: 0.5441557168960571, TEST_Seq_Loss: 0.3771285116672516, Test_Accuracy_Rate: 60.89%, Test_Ones_Portion: 0.6679661273956299
Epoch 39, Total Loss: 1.0423518419265747, Clf Loss: 1.0423518419265747, Filter Loss: 0.5474335551261902, Seq Loss: 0.36768072843551636, Accuracy Rate: 73.83%, Ones Portion: 

Epoch 56, Total Loss: 0.8706278204917908, Clf Loss: 0.8706278204917908, Filter Loss: 0.5515069365501404, Seq Loss: 0.34753498435020447, Accuracy Rate: 77.06%, Ones Portion: 0.7011290788650513,             Test_Total_Loss: 1.863793134689331, Test_Clf_Loss: 1.863793134689331, Test_Filter_Loss: 0.5469055771827698, TEST_Seq_Loss: 0.36402925848960876, Test_Accuracy_Rate: 59.63%, Test_Ones_Portion: 0.6838937997817993
Epoch 57, Total Loss: 1.1838487386703491, Clf Loss: 1.1838487386703491, Filter Loss: 0.5568685531616211, Seq Loss: 0.31944796442985535, Accuracy Rate: 72.94%, Ones Portion: 0.7288641333580017,             Test_Total_Loss: 2.171022891998291, Test_Clf_Loss: 2.171022891998291, Test_Filter_Loss: 0.5688385963439941, TEST_Seq_Loss: 0.2399543821811676, Test_Accuracy_Rate: 58.28%, Test_Ones_Portion: 0.805351734161377
Epoch 58, Total Loss: 1.3611013889312744, Clf Loss: 1.3611013889312744, Filter Loss: 0.5767002701759338, Seq Loss: 0.20075009763240814, Accuracy Rate: 68.38%, Ones Portion:

Epoch 76, Total Loss: 0.8594827055931091, Clf Loss: 0.8594827055931091, Filter Loss: 0.569939136505127, Seq Loss: 0.23632420599460602, Accuracy Rate: 78.45%, Ones Portion: 0.817060649394989,             Test_Total_Loss: 1.868947982788086, Test_Clf_Loss: 1.868947982788086, Test_Filter_Loss: 0.5664564371109009, TEST_Seq_Loss: 0.25159186124801636, Test_Accuracy_Rate: 61.18%, Test_Ones_Portion: 0.8009810447692871
===MODEL1 WEIGHTS SAVED=== ./model/emb_layer_weights
Epoch 77, Total Loss: 0.8481647968292236, Clf Loss: 0.8481647968292236, Filter Loss: 0.5702962875366211, Seq Loss: 0.2361604869365692, Accuracy Rate: 78.69%, Ones Portion: 0.8173043131828308,             Test_Total_Loss: 1.8766013383865356, Test_Clf_Loss: 1.8766013383865356, Test_Filter_Loss: 0.5667061805725098, TEST_Seq_Loss: 0.2508109509944916, Test_Accuracy_Rate: 61.57%, Test_Ones_Portion: 0.8018060326576233
===MODEL1 WEIGHTS SAVED=== ./model/emb_layer_weights
Epoch 78, Total Loss: 0.8297099471092224, Clf Loss: 0.829709947109

...

Epoch 1909, Total Loss: 0.0557711198925972, Clf Loss: 0.0557711198925972, Filter Loss: 0.2300480455160141, Seq Loss: 0.9999778270721436, Accuracy Rate: 95.85%, Ones Portion: 0.0031698746606707573,             Test_Total_Loss: 3.1906607151031494, Test_Clf_Loss: 3.1906607151031494, Test_Filter_Loss: 0.22862322628498077, TEST_Seq_Loss: 0.9999793767929077, Test_Accuracy_Rate: 67.72%, Test_Ones_Portion: 0.003321840660646558
Epoch 1910, Total Loss: 0.05556173622608185, Clf Loss: 0.05556173622608185, Filter Loss: 0.23013417422771454, Seq Loss: 0.999977707862854, Accuracy Rate: 96.09%, Ones Portion: 0.003173187840729952,             Test_Total_Loss: 3.2127935886383057, Test_Clf_Loss: 3.2127935886383057, Test_Filter_Loss: 0.2286461740732193, TEST_Seq_Loss: 0.9999791383743286, Test_Accuracy_Rate: 67.47%, Test_Ones_Portion: 0.0033247489482164383
Epoch 1911, Total Loss: 0.05531805008649826, Clf Loss: 0.05531805008649826, Filter Loss: 0.23025351762771606, Seq Loss: 0.9999779462814331, Accuracy Rate

Epoch 1929, Total Loss: 0.05835094302892685, Clf Loss: 0.05835094302892685, Filter Loss: 0.22985149919986725, Seq Loss: 0.9999783039093018, Accuracy Rate: 96.04%, Ones Portion: 0.0031665002461522818,             Test_Total_Loss: 3.236220359802246, Test_Clf_Loss: 3.236220359802246, Test_Filter_Loss: 0.22832362353801727, TEST_Seq_Loss: 0.999979555606842, Test_Accuracy_Rate: 66.36%, Test_Ones_Portion: 0.003318541683256626
Epoch 1930, Total Loss: 0.05848883464932442, Clf Loss: 0.05848883464932442, Filter Loss: 0.2301143854856491, Seq Loss: 0.9999781847000122, Accuracy Rate: 95.94%, Ones Portion: 0.0031740248668938875,             Test_Total_Loss: 3.2512311935424805, Test_Clf_Loss: 3.2512311935424805, Test_Filter_Loss: 0.2284354269504547, TEST_Seq_Loss: 0.9999797940254211, Test_Accuracy_Rate: 67.09%, Test_Ones_Portion: 0.003312045009806752
Epoch 1931, Total Loss: 0.061878301203250885, Clf Loss: 0.061878301203250885, Filter Loss: 0.22999514639377594, Seq Loss: 0.9999790191650391, Accuracy Ra

Epoch 1949, Total Loss: 0.05660099908709526, Clf Loss: 0.05660099908709526, Filter Loss: 0.23040586709976196, Seq Loss: 0.9999773502349854, Accuracy Rate: 96.10%, Ones Portion: 0.003160197753459215,             Test_Total_Loss: 3.2741599082946777, Test_Clf_Loss: 3.2741599082946777, Test_Filter_Loss: 0.2287304699420929, TEST_Seq_Loss: 0.999979555606842, Test_Accuracy_Rate: 66.99%, Test_Ones_Portion: 0.0032799006439745426
Epoch 1950, Total Loss: 0.056379929184913635, Clf Loss: 0.056379929184913635, Filter Loss: 0.2302195429801941, Seq Loss: 0.9999784231185913, Accuracy Rate: 96.03%, Ones Portion: 0.003108665579929948,             Test_Total_Loss: 3.271113872528076, Test_Clf_Loss: 3.271113872528076, Test_Filter_Loss: 0.2286028116941452, TEST_Seq_Loss: 0.9999796748161316, Test_Accuracy_Rate: 67.09%, Test_Ones_Portion: 0.0032765932846814394
Epoch 1951, Total Loss: 0.055544327944517136, Clf Loss: 0.055544327944517136, Filter Loss: 0.23024028539657593, Seq Loss: 0.9999784231185913, Accuracy R

Epoch 1969, Total Loss: 0.05566256865859032, Clf Loss: 0.05566256865859032, Filter Loss: 0.2291678935289383, Seq Loss: 0.9999781847000122, Accuracy Rate: 95.92%, Ones Portion: 0.003119658213108778,             Test_Total_Loss: 3.286983013153076, Test_Clf_Loss: 3.286983013153076, Test_Filter_Loss: 0.22744762897491455, TEST_Seq_Loss: 0.9999800324440002, Test_Accuracy_Rate: 67.23%, Test_Ones_Portion: 0.003289637854322791
Epoch 1970, Total Loss: 0.055302686989307404, Clf Loss: 0.055302686989307404, Filter Loss: 0.2289365977048874, Seq Loss: 0.9999781847000122, Accuracy Rate: 96.03%, Ones Portion: 0.0031144041568040848,             Test_Total_Loss: 3.300809860229492, Test_Clf_Loss: 3.300809860229492, Test_Filter_Loss: 0.22715915739536285, TEST_Seq_Loss: 0.9999797940254211, Test_Accuracy_Rate: 66.94%, Test_Ones_Portion: 0.003290012711659074
Epoch 1971, Total Loss: 0.05544145032763481, Clf Loss: 0.05544145032763481, Filter Loss: 0.22893774509429932, Seq Loss: 0.9999779462814331, Accuracy Rate

Epoch 1989, Total Loss: 0.055437956005334854, Clf Loss: 0.055437956005334854, Filter Loss: 0.22332307696342468, Seq Loss: 0.9999781847000122, Accuracy Rate: 95.79%, Ones Portion: 0.00311732548289001,             Test_Total_Loss: 3.3231656551361084, Test_Clf_Loss: 3.3231656551361084, Test_Filter_Loss: 0.22194674611091614, TEST_Seq_Loss: 0.9999797940254211, Test_Accuracy_Rate: 67.52%, Test_Ones_Portion: 0.003283059224486351
Epoch 1990, Total Loss: 0.05565162003040314, Clf Loss: 0.05565162003040314, Filter Loss: 0.22335895895957947, Seq Loss: 0.9999784231185913, Accuracy Rate: 96.15%, Ones Portion: 0.003115415573120117,             Test_Total_Loss: 3.3286876678466797, Test_Clf_Loss: 3.3286876678466797, Test_Filter_Loss: 0.22181399166584015, TEST_Seq_Loss: 0.9999797940254211, Test_Accuracy_Rate: 67.28%, Test_Ones_Portion: 0.0032839542254805565
Epoch 1991, Total Loss: 0.055221423506736755, Clf Loss: 0.055221423506736755, Filter Loss: 0.22305922210216522, Seq Loss: 0.9999783039093018, Accura

In [523]:
# model1.save('./model/emb_layer') #tf.keras.models.load_model
tf.saved_model.save(model1,'./model/emb_layer')
model3 = tf.keras.models.load_model('./model/emb_layer')

TypeError: unsupported callable

In [524]:
model3 = model1.load_weights('./model/emb_layer_weight')

AttributeError: 'CheckpointLoadStatus' object has no attribute 'summary'

In [527]:
model1.emb.set_weights(model3.emb.get_weights())

In [526]:
model3 = model1
model3.load_weights('./model/emb_layer_weight')
model3.summary()

Model: "base_model_1_58"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_58 (Masking)         multiple                  0         
_________________________________________________________________
glove_emb (Embedding)        multiple                  2138112   
_________________________________________________________________


ValueError: You tried to call `count_params` on common_extract, but the layer isn't built. You can build it manually via: `common_extract.build(batch_input_shape)`.

* 在某些參數的情況下0 1是train不起來的但是weight可以
    * alpha = 0.0, beta=0.6
    * init_w = tensorflow.keras.initializers.Constant(value=0.9), init_b = tensorflow.keras.initializers.Constant(value=0.7)
* 0/1 with emb比較容易overfit。如果是weight的比較沒那麼嚴重

* 不用sigmoid或是hard_sigmoid。改良relu + linear，並拆成兩個model，把round前面多加上clip
    * sigmoid中間的變化太快(一瞬間就會掉到0或是1)，改成relu在>0~無限大(linear為了還在0~1)再去clip再round，可以看到每個epoch的變化
* 1st phase的beta一定要>=0.6否則不會動，ones#都會是0
    * beta=1 (放0 1進去): weight init設成1也沒用，但把bias設成1就會一開始都是ones#=1了。weight=1 bias=0.2都匯市0，bias=0.3會是0.87。0.6/0.3都是0。0.8/0.3差不多是0.5(但是train很慢acc進步很慢就是了)。0.5/0.5是從0.01開始往上升 (0 1放進去會比較難train是因為它的變化量太大，一下就是有或沒有，所以clf可能學不好，但如果是weight每次gradient進步的都是一小點就會比較容易上升)
    * beta=0.6 (放0 1進去): 0.8/0.5都是0。0.9/0.8 從0.9一直到0。0.9/0.6差不多是從0.5但又有時候到0.7都是0.0(很難train，Nadam換個opt有時候沒用。EX變成adam 0.9/0.8才有0.96開始但如果0.9/0.75變成0開始。Rmsprop 0.9/0.8又是從0.95開始往下)。但如果都改成傳入weight就都沒問題。ones一開始大概0.5 weight平均，也不會卡住
* 建議: 先訓練embedding weight matrix，但是要看goal是要怎樣的matrix
* 其實他不管幾%都會train得很好，除非固定embedding，或是用更弱的clf
* 若設定alpha，就像是regularizer term (penalty)，設越大drop越多
* 一開始先很多ones，再越來越少個

* 同一個opt若加入transform就會train不起來
* 兩個不同的opt加入transform也會train不起來 (persistent、non-persis都不行)，且與BN無關